# Getting Started with Snowpark Connect for Apache Spark

Welcome to Snowpark Connect for Apache Spark!

This notebook demonstrates how to run familiar PySpark workloads directly on Snowflake
using the Tasty Bytes food truck dataset. You'll learn:
- Environment setup
- PySpark DataFrame API fundamentals
- Data exploration and transformation capabilities
- Performance benefits and limitations
- Best practices for Spark-on-Snowflake workflows

Let's begin!

In [ ]:
# Environment Setup - TODO REPLACE WITH ACTUAL IMPORTS
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Initialize Spark session for Snowflake
spark = SparkSession.builder \
    .appName("TastyBytes-PySpark-Demo") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Connection parameters (pre-configured in learning environment)
print("✅ Spark session initialized successfully!")
print(f"Spark version: {spark.version}")

In [ ]:
# Load Tasty Bytes core tables - TODO - UPDATE WITH ACTUAL TABLES
order_header = spark.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_HEADER")
order_detail = spark.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_DETAIL") 
menu = spark.table("FROSTBYTE_TASTY_BYTES.RAW_POS.MENU")
truck = spark.table("FROSTBYTE_TASTY_BYTES.RAW_POS.TRUCK")
location = spark.table("FROSTBYTE_TASTY_BYTES.RAW_POS.LOCATION")

# Schema exploration - PySpark's power in data discovery
print("📊 ORDER_HEADER Schema:")
order_header.printSchema()

print("\n📈 Dataset sizes:")
tables = [
    ("ORDER_HEADER", order_header),
    ("ORDER_DETAIL", order_detail),
    ("MENU", menu),
    ("TRUCK", truck),
    ("LOCATION", location)
]

for name, df in tables:
    count = df.count()
    print(f"{name}: {count:,} rows")

In [ ]:

# TODO update with appropriate APIs and string literals
# SELECT operations - Column selection and aliasing
sales_summary = order_header.select(
    col("ORDER_ID"),
    col("ORDER_TS").alias("order_timestamp"),
    col("ORDER_AMOUNT").alias("total_amount"),
    col("TRUCK_ID"),
    col("LOCATION_ID")
)

# FILTER operations - Data filtering with multiple conditions
recent_orders = sales_summary.filter(
    (col("order_timestamp") >= "2022-01-01") & 
    (col("total_amount") > 10.0)
)

# AGGREGATION operations - GroupBy and statistical functions
daily_sales = order_header \
    .withColumn("order_date", to_date(col("ORDER_TS"))) \
    .groupBy("order_date") \
    .agg(
        count("ORDER_ID").alias("total_orders"),
        sum("ORDER_AMOUNT").alias("daily_revenue"),
        avg("ORDER_AMOUNT").alias("avg_order_value"),
        max("ORDER_AMOUNT").alias("max_order"),
        min("ORDER_AMOUNT").alias("min_order")
    ) \
    .orderBy(desc("order_date"))

print("📅 Daily Sales Performance:")
daily_sales.show(10)

In [ ]:
# TODO - add actual APIs
# Advanced Spark Transformations
# WINDOW FUNCTIONS - Advanced analytics capabilities
window_spec = Window.partitionBy("TRUCK_ID").orderBy("ORDER_TS")

truck_performance = order_header \
    .withColumn("running_total", sum("ORDER_AMOUNT").over(window_spec)) \
    .withColumn("order_rank", row_number().over(window_spec)) \
    .withColumn("prev_order_amount", lag("ORDER_AMOUNT", 1).over(window_spec))

# JOINS - Combining multiple datasets
enriched_orders = order_header \
    .join(truck, "TRUCK_ID") \
    .join(location, "LOCATION_ID") \
    .select(
        col("ORDER_ID"),
        col("ORDER_TS"),
        col("ORDER_AMOUNT"),
        col("TRUCK_BRAND_NAME"),
        col("PRIMARY_CITY"),
        col("REGION"),
        col("COUNTRY")
    )

print("🚚 Enriched Orders with Truck and Location Data:")
enriched_orders.show(5)

In [ ]:
# Complex Analytics - Menu Item Performance
# TODO - update with actual APIs and string literals
# Complex multi-table analysis
menu_performance = order_detail \
    .join(order_header, "ORDER_ID") \
    .join(menu, "MENU_ITEM_ID") \
    .groupBy("MENU_ITEM_NAME", "ITEM_CATEGORY") \
    .agg(
        count("ORDER_DETAIL_ID").alias("times_ordered"),
        sum("QUANTITY").alias("total_quantity"),
        sum("PRICE").alias("total_revenue"),
        avg("PRICE").alias("avg_price")
    ) \
    .withColumn("revenue_per_order", col("total_revenue") / col("times_ordered")) \
    .orderBy(desc("total_revenue"))

print("🍔 Top Menu Items by Revenue:")
menu_performance.show(15)

# Time-based analysis with date functions
seasonal_trends = order_header \
    .withColumn("month", month(col("ORDER_TS"))) \
    .withColumn("day_of_week", dayofweek(col("ORDER_TS"))) \
    .withColumn("hour", hour(col("ORDER_TS"))) \
    .groupBy("month", "day_of_week") \
    .agg(
        count("ORDER_ID").alias("order_count"),
        avg("ORDER_AMOUNT").alias("avg_amount")
    ) \
    .orderBy("month", "day_of_week")

print("📈 Seasonal Ordering Patterns:")
seasonal_trends.show()

In [ ]:
# PySpark SQL Integration
# TODO - verfiy sql and apis
# Register DataFrames as temporary views for SQL access
order_header.createOrReplaceTempView("orders")
order_detail.createOrReplaceTempView("order_items")
menu.createOrReplaceTempView("menu_items")
truck.createOrReplaceTempView("trucks")

# Complex SQL queries using Spark SQL
top_performing_trucks = spark.sql("""
    SELECT 
        t.TRUCK_BRAND_NAME,
        t.TRUCK_ID,
        COUNT(o.ORDER_ID) as total_orders,
        SUM(o.ORDER_AMOUNT) as total_revenue,
        AVG(o.ORDER_AMOUNT) as avg_order_value,
        RANK() OVER (ORDER BY SUM(o.ORDER_AMOUNT) DESC) as revenue_rank
    FROM orders o
    JOIN trucks t ON o.TRUCK_ID = t.TRUCK_ID
    WHERE o.ORDER_TS >= '2022-01-01'
    GROUP BY t.TRUCK_BRAND_NAME, t.TRUCK_ID
    ORDER BY total_revenue DESC
    LIMIT 10
""")

print("🏆 Top Performing Food Trucks:")
top_performing_trucks.show()

In [ ]:
# Performance Optimization Techniques
# TODO - validate with eng and product
# Caching for iterative analysis
daily_sales.cache()
print("💾 Cached daily_sales DataFrame for reuse")

# Partitioning awareness
partitioned_orders = order_header \
    .withColumn("year_month", date_format(col("ORDER_TS"), "yyyy-MM")) \
    .repartition(col("year_month"))

# Broadcast joins for small dimension tables
from pyspark.sql.functions import broadcast

# Efficient join with small lookup table
efficient_join = order_header \
    .join(broadcast(truck), "TRUCK_ID") \
    .select("ORDER_ID", "ORDER_AMOUNT", "TRUCK_BRAND_NAME")

print("⚡ Applied performance optimizations")

## Limitations and Workarounds

1. **Unsupported Data Types:**
   - DayTimeIntervalType
   - YearMonthIntervalType  
   - UserDefinedTypes

2. **Performance Considerations:**
   - Some operations may not push down to Snowflake engine
   - Complex UDFs might impact performance
   - Large shuffles can be expensive

3. **Feature Gaps:**
   - Some advanced Spark features may not be available
   - Third-party library limitations
   - Streaming operations not supported

4. **Best Practices:**
   - Prefer SQL operations when possible
   - Use broadcast joins for small tables
   - Cache frequently accessed DataFrames
   - Minimize data movement between operations

In [ ]:
# Example of operation that might not optimize well
try:
    # Complex Python UDF (may not push down efficiently)
    from pyspark.sql.functions import udf
    
    @udf(returnType=StringType())
    def complex_categorization(amount):
        if amount < 10:
            return "Small"
        elif amount < 25:
            return "Medium"
        else:
            return "Large"
    
    categorized_orders = order_header \
        .withColumn("order_size", complex_categorization(col("ORDER_AMOUNT")))
    
    print("✅ UDF applied successfully (but may impact performance)")
    
except Exception as e:
    print(f"❌ UDF limitation encountered: {e}")

## Migration from Traditional Spark to Snowpark Connect"

**Code Changes Required:**
- Minimal changes to existing PySpark code
- Update connection configuration
- Review UDF implementations
- Test performance characteristics

**Benefits:**
✅ No Spark cluster management
✅ Snowflake's security and governance
✅ Elastic scaling
✅ Integrated with Snowflake ecosystem

**Migration Checklist:**
□ Inventory existing Spark jobs
□ Identify unsupported features
□ Test performance on representative workloads
□ Update deployment processes
□ Train team on new architecture

## 🎯 Next Steps for Learning

1. **Explore Advanced Features:**
   - Machine Learning with MLlib
   - Streaming data processing
   - Graph analytics

2. **Performance Tuning:**
   - Query optimization techniques
   - Partitioning strategies
   - Caching best practices

3. **Integration Patterns:**
   - Snowflake native features
   - External data sources
   - Orchestration with Airflow

4. **Production Deployment:**
   - CI/CD pipelines
   - Monitoring and alerting
   - Error handling strategies

# Cleanup
spark.catalog.clearCache()
print("🧹 Cleaned up cached data")